# Train a LaTeX OCR model
In this brief notebook I show how you can finetune/train an OCR model.

I've opted to mix in handwritten data into the regular pdf LaTeX images. For that I started out with the released pretrained model and continued training on the slightly larger corpus.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Thu Jan 12 07:49:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#make data

##cài đặt kaggel

In [3]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
! mkdir ~/.kaggle

In [5]:
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

## data train

In [7]:
!kaggle datasets download -d dtrnngc/data-scio

100% 1.02G/1.02G [00:52<00:00, 20.4MB/s]
100% 1.02G/1.02G [00:52<00:00, 20.8MB/s]


In [8]:
!unzip /content/data-scio.zip

Streaming output truncated to the last 5000 lines.
  inflating: datareal/1110/normed_1110_550.jpg  
  inflating: datareal/1110/normed_1110_5500.jpg  
  inflating: datareal/1110/normed_1110_5501.jpg  
  inflating: datareal/1110/normed_1110_5502.jpg  
  inflating: datareal/1110/normed_1110_5503.jpg  
  inflating: datareal/1110/normed_1110_5504.jpg  
  inflating: datareal/1110/normed_1110_5505.jpg  
  inflating: datareal/1110/normed_1110_5506.jpg  
  inflating: datareal/1110/normed_1110_5507.jpg  
  inflating: datareal/1110/normed_1110_5508.jpg  
  inflating: datareal/1110/normed_1110_5509.jpg  
  inflating: datareal/1110/normed_1110_551.jpg  
  inflating: datareal/1110/normed_1110_5510.jpg  
  inflating: datareal/1110/normed_1110_5511.jpg  
  inflating: datareal/1110/normed_1110_5512.jpg  
  inflating: datareal/1110/normed_1110_5513.jpg  
  inflating: datareal/1110/normed_1110_5514.jpg  
  inflating: datareal/1110/normed_1110_5515.jpg  
  inflating: datareal/1110/normed_1110_5516.jpg  
 

## data val

In [9]:
!kaggle datasets download -d dtrnngc/private-test
!unzip /content/private-test.zip

 65% 1.00M/1.54M [00:00<00:00, 1.28MB/s]
100% 1.54M/1.54M [00:00<00:00, 1.79MB/s]
Archive:  /content/private-test.zip
  inflating: images_private_test/images_private_test/part4.1_10004.jpg  
  inflating: images_private_test/images_private_test/part4.1_10008.jpg  
  inflating: images_private_test/images_private_test/part4.1_10035.jpg  
  inflating: images_private_test/images_private_test/part4.1_10084.jpg  
  inflating: images_private_test/images_private_test/part4.1_10088.jpg  
  inflating: images_private_test/images_private_test/part4.1_10137.jpg  
  inflating: images_private_test/images_private_test/part4.1_10141.jpg  
  inflating: images_private_test/images_private_test/part4.1_10165.jpg  
  inflating: images_private_test/images_private_test/part4.1_10202.jpg  
  inflating: images_private_test/images_private_test/part4.1_10225.jpg  
  inflating: images_private_test/images_private_test/part4.1_10269.jpg  
  inflating: images_private_test/images_private_test/part4.1_10303.jpg  
  infl

In [10]:
!kaggle datasets download -d aidapearson/ocr-data

100% 10.1G/10.1G [07:59<00:00, 22.1MB/s]
100% 10.1G/10.1G [07:59<00:00, 22.6MB/s]


In [ ]:
!unzip /content/ocr-data.zip

In [12]:
!rm /content/ocr-data.zip

In [60]:
DIR = "/content"
import json
import pandas as pd

df_append = pd.DataFrame()

def create_data_frame(raw_data):
    data = {}
    data["image"] = []
    data["formula"] = []
    
    for image in raw_data:
        data["image"].append(image["filename"])
        data["formula"].append(image["latex"])
        
    df = pd.DataFrame.from_dict(data)
    return df

print("Loading batch overview.")
for i in range(1,11):
  file = open(f"{DIR}/batch_{i}/JSON/kaggle_data_{i}.json")
  data = json.load(file)
  df = create_data_frame(data)
  df.to_csv(f"Aida_data{i}.csv", index=False)

Loading batch overview.


In [62]:
df_append = pd.DataFrame()
#append all files together
for i in range(1,11):
  df_temp = pd.read_csv(f"/content/Aida_data{i}.csv")
  df_append = df_append.append(df_temp, ignore_index=True)
df_append.to_csv("/content/Aida_data.csv")

In [59]:
df_append.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame

# resize image

In [15]:
!pip install blend_modes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for blend_modes: filename=blend_modes-2.1.0-py3-none-any.whl size=9554 sha256=63d175af28444e083d1a4ece9605dd93e3e37f33ad14a6030f105464d8cf72e6
  Stored in directory: /root/.cache/pip/wheels/75/3a/3b/d64d7701def9e36eab649537db887433bd8921ec568634b42a
Successfully built blend_modes


In [25]:
from PIL import Image
from typing import Tuple
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
from regex import W
import pandas as pd
import tqdm
import blend_modes as blm
def minmax_size(img, max_dimensions: Tuple[int, int] = None, min_dimensions: Tuple[int, int] = None):
    """Resize or pad an image to fit into given dimensions

    Args:
        img (Image): Image to scale up/down.
        max_dimensions (Tuple[int, int], optional): Maximum dimensions. Defaults to None.
        min_dimensions (Tuple[int, int], optional): Minimum dimensions. Defaults to None.

    Returns:
        Image: Image with correct dimensionality
    """
    if max_dimensions is not None:
        ratios = [a/b for a, b in zip(img.size, max_dimensions)] #44,672 ratio: tỉ lệ gữa size ảnh thật và size max
        if any([r > 1 for r in ratios]):
            size = np.array(img.size)//max(ratios) #(44,14)
            img = img.resize(size.astype(int), Image.BILINEAR)

    if min_dimensions is not None:
        # hypothesis: there is a dim in img smaller than min_dimensions, and return a proper dim >= min_dimensions
        padded_size = [max(img_dim, min_dim) for img_dim, min_dim in zip(img.size, min_dimensions)] #(w, h)
        if padded_size != list(img.size):  # assert hypothesis
            x = (padded_size[0]- img.size[0])//2
            y = (padded_size[1]- img.size[1])//2
            padded_im = Image.new('RGB', padded_size, color = 'white')
            padded_im.paste(img,(x,y))
            img = padded_im
    return img


def div32(img):
    w, h = img.size
    w = (w//32 + 1)*32 if w%32!=0 else w
    h = (h//32 + 1)*32 if h%32!=0 else h
    x = (w- img.size[0])//2
    y = (h- img.size[1])//2
    padded_im = Image.new('RGB', (w, h), color = 'white')
    padded_im.paste(img,(x,y))
    img = padded_im
    return img

def resize_image(path_folder, target_folder, random_scale = False, scale = 1):
    for path in tqdm.tqdm(glob.glob(path_folder)):
      if random_scale == True:
        scale = np.random.choice([0.6, 0.5, 0.7,0.8, 0.9, 1, 1.1, 1.2])
      else:
        scale = scale
      img = Image.open(path)
      w, h = img.size
      w = w*scale
      h = h*scale
      img = img.resize((round(w), round(h)))
      img = div32(minmax_size(img, (672, 192), (32, 32)))
      file_name =  path.split("/")[-1]
      img.save(target_folder + file_name)

def resize_image_csv(path_csv, root_path):
    myFr = pd.read_csv(path_csv)
    images = []
    formulas = []
    for path, formula in tqdm.tqdm(zip(myFr["image"], myFr["formula"])):
        paths = root_path + path
        img = Image.open(paths)
        w, h = img.size
        for k in range(3):
            scale = np.random.choice([0.5, 0.7, 0.9, 1.0, 1.2])
            w = w*scale
            h = h*scale
            imgs = img.resize((round(w), round(h)))
            imgs = div32(minmax_size(img, (672, 192), (32, 32)))
            imgs.save(f"/content/dataRealResize/{k}" + path)
            images.append(f"{k}"+path)
            formulas.append(formula)
    myFR = pd.DataFrame({"image":images,"formula":formulas})
    myFR.to_csv("data_real_resize.csv", index = False)


def resize4test(path, scale):
    img = Image.open(path)
    w, h = img.size
    w = w*scale
    h = h*scale
    img = img.resize((round(w), round(h)))
    img = div32(minmax_size(img, (672, 192), (32, 32)))
    return img

def resize_handwriting(path_folder, scale ,target_folder):
    for path in tqdm.tqdm(glob.glob(path_folder)):
      # print(path)
      img_in = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGBA).astype(float)/255
      w, h = img_in.shape[1], img_in.shape[0]
      ratio = w/h
      w = int(w*scale)
      h = int(h*scale)
      if h < 8:
        h = 8
        w = int(ratio*h)
      img_in = cv2.resize(img_in,(w, h))
      i = np.random.randint(1,6)
      img_layer = cv2.cvtColor(cv2.imread(f"/content/bg{i}.png"), cv2.COLOR_BGR2RGBA).astype(float)/255
      img_layer = cv2.resize(img_layer,(w, h))
      img_out = blm.darken_only(img_in,img_layer,1.)
      img_out = cv2.cvtColor((img_out*255).astype(np.uint8), cv2.COLOR_BGRA2RGB)
      filename = path.split("/")[-1]
      cv2.imwrite(target_folder + filename,img_out)




In [48]:
resize_image(r"/content/data_train/data_train/*.png", target_folder = "/content/data_train/data_train/", random_scale=False)

100%|██████████| 134663/134663 [03:11<00:00, 703.04it/s]


In [49]:
%mkdir dataRealResize

In [50]:
# resize_image(r"/content/datareal/1110/*", random_scale=True)
resize_image_csv("/content/data_real.csv","/content/datareal/1110/")

11216it [00:26, 420.53it/s]


In [51]:
%mkdir HandwriteFormula2

In [76]:
resize_handwriting("/content/HandwriteFormula/CROHME/*", 0.4, "/content/HandwriteFormula2/")

100%|██████████| 10822/10822 [01:01<00:00, 175.71it/s]


In [77]:
resize_image("/content/HandwriteFormula2/*", "/content/HandwriteFormula2/")

100%|██████████| 10822/10822 [00:37<00:00, 290.99it/s]


In [18]:
%mkdir AidaImage

In [54]:
resize_image("/content/*/background_images/*", "/content/AidaImage/", scale=0.1)

100%|██████████| 100000/100000 [22:06<00:00, 75.40it/s]


In [93]:
!pip install pix2tex[train] -qq
!pip install gpustat -q
!pip install opencv-python-headless==4.1.2.30 -U -q
!pip install --upgrade --no-cache-dir gdown -q
# If using wandb
!pip install -q wandb 
# you can cancel this if you don't wan't to use it or don't have a W&B acc.
#!wandb login

# edit file dataset

In [27]:
# from google.colab import files
# files.view('/usr/local/lib/python3.7/dist-packages/pix2tex/dataset/dataset.py')

In [28]:
%%writefile /usr/local/lib/python3.8/dist-packages/pix2tex/dataset/dataset.py
import torch
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import imagesize
import logging
import glob
import os
from os.path import join
from collections import defaultdict
import pickle
import cv2
from transformers import PreTrainedTokenizerFast
from tqdm.auto import tqdm

from pix2tex.utils.utils import in_model_path
from pix2tex.dataset.transforms import train_transform, test_transform


import pandas as pd


class Im2LatexDataset:
    keep_smaller_batches = False
    shuffle = True
    batchsize = 16
    max_dimensions = (1024, 512)
    min_dimensions = (32, 32)
    max_seq_len = 1024
    pad_token = "[PAD]"
    bos_token = "[BOS]"
    eos_token = "[EOS]"
    pad_token_id = 0
    bos_token_id = 1
    eos_token_id = 2
    transform = train_transform
    data = defaultdict(lambda: [])

    def __init__(self, equations=None, images=None, tokenizer=None, shuffle=True, batchsize=16, max_seq_len=1024,
                 max_dimensions=(1024, 512), min_dimensions=(32, 32), pad=False, keep_smaller_batches=False, test=False):
        """Generates a torch dataset from pairs of `equations` and `images`.

        Args:
            equations (str, optional): Path to equations. Defaults to None.
            images (str, optional): Directory where images are saved. Defaults to None.
            tokenizer (str, optional): Path to saved tokenizer. Defaults to None.
            shuffle (bool, opitonal): Defaults to True. 
            batchsize (int, optional): Defaults to 16.
            max_seq_len (int, optional): Defaults to 1024.
            max_dimensions (tuple(int, int), optional): Maximal dimensions the model can handle
            min_dimensions (tuple(int, int), optional): Minimal dimensions the model can handle
            pad (bool): Pad the images to `max_dimensions`. Defaults to False.
            keep_smaller_batches (bool): Whether to also return batches with smaller size than `batchsize`. Defaults to False.
            test (bool): Whether to use the test transformation or not. Defaults to False.
        """
        print("start make file dataset.pkl")
        if images is not None and equations is not None:
            assert tokenizer is not None
            myFrame = pd.read_csv(equations)
            self.images = myFrame["image"]
            self.sample_size = len(self.images)
            eqs = myFrame["formula"]
            self.imgPath = images
            self.tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer)
            self.shuffle = shuffle
            self.batchsize = batchsize
            self.max_seq_len = max_seq_len
            self.max_dimensions = max_dimensions
            self.min_dimensions = min_dimensions
            self.pad = pad
            self.keep_smaller_batches = keep_smaller_batches
            self.test = test
            # check the image dimension for every image and group them together
            try:
                print("make dataset")
                for i, im in tqdm(enumerate(self.images), total=len(self.images)):
                    path = images + im
                    width, height = imagesize.get(path)
                    if min_dimensions[0] <= width <= max_dimensions[0] and min_dimensions[1] <= height <= max_dimensions[1]:
                        self.data[(width, height)].append((eqs[i], path))
                print("success!")
            except KeyboardInterrupt:
                pass
            self.data = dict(self.data)
            self._get_size()

            iter(self)

    def __len__(self):
        return self.size

    def __iter__(self):
        # dùng để chia batch
        self.i = 0
        self.transform = test_transform if self.test else train_transform
        self.pairs = []
        for k in self.data:
            info = np.array(self.data[k], dtype=object)
            p = torch.randperm(len(info)) if self.shuffle else torch.arange(len(info))
            for i in range(0, len(info), self.batchsize):
                batch = info[p[i:i+self.batchsize]]
                if len(batch.shape) == 1:
                    batch = batch[None, :]
                if len(batch) < self.batchsize and not self.keep_smaller_batches:
                    continue
                self.pairs.append(batch)
        if self.shuffle:
            self.pairs = np.random.permutation(np.array(self.pairs, dtype=object))
        else:
            self.pairs = np.array(self.pairs, dtype=object)
        self.size = len(self.pairs)
        return self

    def __next__(self):
        # dùng để load batch
        if self.i >= self.size:
            raise StopIteration
        self.i += 1
        return self.prepare_data(self.pairs[self.i-1])

    def prepare_data(self, batch):
        """loads images into memory

        Args:
            batch (numpy.array[[str, str]]): array of equations and image path pairs

        Returns:
            tuple(torch.tensor, torch.tensor): data in memory
        """

        eqs, ims = batch.T
        tok = self.tokenizer(list(eqs), return_token_type_ids=False)
        
        # pad with bos and eos token
        for k, p in zip(tok, [[self.bos_token_id, self.eos_token_id], [1, 1]]):
            tok[k] = pad_sequence([torch.LongTensor([p[0]]+x+[p[1]]) for x in tok[k]], batch_first=True, padding_value=self.pad_token_id)
        # check if sequence length is too long
        if self.max_seq_len < tok['attention_mask'].shape[1]:
            return next(self)
        images = []
        image_path = []
        for path in list(ims):
            im = cv2.imread(path)
            if im is None:
                print(path, 'not found!')
                continue
            im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            if not self.test:
                # sometimes convert to bitmask
                if np.random.random() < .04:
                    im[im != 255] = 0
            images.append(self.transform(image=im)['image'][:1])
            image_path.append(path)
        try:
            images = torch.cat(images).float().unsqueeze(1)
        except RuntimeError:
            logging.critical('Images not working: %s' % (' '.join(list(ims))))
            return None, None, None
        if self.pad:
            h, w = images.shape[2:]
            images = F.pad(images, (0, self.max_dimensions[0]-w, 0, self.max_dimensions[1]-h), value=1)
        return tok, images, image_path

    def _get_size(self):
        self.size = 0
        for k in self.data:
            div, mod = divmod(len(self.data[k]), self.batchsize)
            self.size += div  # + (1 if mod > 0 else 0)

    def load(self, filename, args=[]):
        """returns a pickled version of a dataset

        Args:
            filename (str): Path to dataset
        """
        if not os.path.exists(filename):
            with in_model_path():
                tempf = os.path.join('..', filename)
                if os.path.exists(tempf):
                    filename = os.path.realpath(tempf)
        with open(filename, 'rb') as file:
            x = pickle.load(file)
        return x

    def combine(self, x):
        """Combine Im2LatexDataset with another Im2LatexDataset

        Args:
            x (Im2LatexDataset): Dataset to absorb
        """
        for key in x.data.keys():
            if key in self.data.keys():
                self.data[key].extend(x.data[key])
                self.data[key] = list(set(self.data[key]))
            else:
                self.data[key] = x.data[key]
        self._get_size()
        iter(self)

    def save(self, filename):
        """save a pickled version of a dataset

        Args:
            filename (str): Path to dataset
        """
        with open(filename, 'wb') as file:
            pickle.dump(self, file)

    def update(self, **kwargs):
        for k in ['batchsize', 'shuffle', 'pad', 'keep_smaller_batches', 'test', 'max_seq_len']:
            if k in kwargs:
                setattr(self, k, kwargs[k])
        if 'max_dimensions' in kwargs or 'min_dimensions' in kwargs:
            if 'max_dimensions' in kwargs:
                self.max_dimensions = kwargs['max_dimensions']
            if 'min_dimensions' in kwargs:
                self.min_dimensions = kwargs['min_dimensions']
            temp = {}
            for k in self.data:
                if self.min_dimensions[0] <= k[0] <= self.max_dimensions[0] and self.min_dimensions[1] <= k[1] <= self.max_dimensions[1]:
                    temp[k] = self.data[k]
            self.data = temp
        if 'tokenizer' in kwargs:
            tokenizer_file = kwargs['tokenizer']
            if not os.path.exists(tokenizer_file):
                with in_model_path():
                    tokenizer_file = os.path.realpath(tokenizer_file)
            self.tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_file)
        self._get_size()
        iter(self)

def generate_tokenizer(equations, output, vocab_size):
    from tokenizers import Tokenizer, pre_tokenizers
    from tokenizers.models import BPE
    from tokenizers.trainers import BpeTrainer
    tokenizer = Tokenizer(BPE())
    tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
    trainer = BpeTrainer(special_tokens=["[PAD]", "[BOS]", "[EOS]"], vocab_size=vocab_size, show_progress=True)
    tokenizer.train(equations, trainer)
    tokenizer.save(path=output, pretty=False)


if __name__ == '__main__':
    # dataloader = Im2LatexDataset(equations="D:\LaTeX-OCR-1\datasets\im2latex_train.csv", images="D:\LaTeX-OCR-1\datasets\\formula_images_processed\\formula_images_processed/", tokenizer="D:\LaTeX-OCR-1\pix2tex\model\dataset\\tokenizer.json")
    # tok, image = next(iter(dataloader))
    # print(tok)
    # print(type(image))
    import argparse
    parser = argparse.ArgumentParser(description='Train model', add_help=False)
    parser.add_argument('-i', '--images', type=str, nargs='+', default=None, help='Image folders')
    parser.add_argument('-e', '--equations', type=str, nargs='+', default=None, help='equations text files')
    parser.add_argument('-t', '--tokenizer', default=None, help='Pretrained tokenizer file')
    parser.add_argument('-o', '--out', type=str, required=True, help='output file')
    parser.add_argument('-s', '--vocab-size', default=8000, type=int, help='vocabulary size when training a tokenizer')
    args = parser.parse_args()
    if args.tokenizer is None:
        with in_model_path():
            args.tokenizer = os.path.realpath(os.path.join('dataset', 'tokenizer.json'))
    if args.images is None and args.equations is not None:
        print('Generate tokenizer')
        generate_tokenizer(args.equations, args.out, args.vocab_size)
    elif args.images is not None and args.equations is not None:
        print('Generate dataset')
        dataset = None
        for images, equations in zip(args.images, args.equations):
            if dataset is None:
                dataset = Im2LatexDataset(equations, images, args.tokenizer)
            else:
                dataset.combine(Im2LatexDataset(equations, images, args.tokenizer))
        dataset.update(batchsize=1, keep_smaller_batches=True)
        dataset.save(args.out)
    else:
        print('Not defined')



Overwriting /usr/local/lib/python3.8/dist-packages/pix2tex/dataset/dataset.py


#edit file eval

In [29]:
# from google.colab import files
# files.view('/usr/local/lib/python3.7/dist-packages/pix2tex/eval.py')

In [30]:
%%writefile /usr/local/lib/python3.8/dist-packages/pix2tex/eval.py
from pix2tex.dataset.dataset import Im2LatexDataset
import argparse
import logging
import yaml

import numpy as np
import torch
from torchtext.data import metrics
from munch import Munch
from tqdm.auto import tqdm
import wandb
from Levenshtein import distance

from pix2tex.models import get_model, Model
from pix2tex.utils import *



def detokenize(tokens, tokenizer):
    toks = [tokenizer.convert_ids_to_tokens(tok) for tok in tokens]
    for b in range(len(toks)):
        for i in reversed(range(len(toks[b]))):
            if toks[b][i] is None:
                toks[b][i] = ''
            toks[b][i] = toks[b][i].replace('Ġ', ' ').strip()
            if toks[b][i] in (['[BOS]', '[EOS]', '[PAD]']):
                del toks[b][i]
    return toks


@torch.no_grad()
def evaluate(model: Model, dataset: Im2LatexDataset, args: Munch, num_batches: int = None, name: str = 'test'):
    """evaluates the model. Returns bleu score on the dataset

    Args:
        model (torch.nn.Module): the model
        dataset (Im2LatexDataset): test dataset
        args (Munch): arguments
        num_batches (int): How many batches to evaluate on. Defaults to None (all batches).
        name (str, optional): name of the test e.g. val or test for wandb. Defaults to 'test'.

    Returns:
        Tuple[float, float, float]: BLEU score of validation set, normed edit distance, token accuracy
    """
    assert len(dataset) > 0
    device = args.device
    log = {}
    bleus, edit_dists, token_acc = [], [], []
    bleu_score, edit_distance, token_accuracy = 0, 1, 0
    pbar = tqdm(enumerate(iter(dataset)), total=len(dataset))
    for i, (seq, im, _) in pbar:
        if seq is None or im is None:
            continue
        #loss = decoder(tgt_seq, mask=tgt_mask, context=encoded)
        dec = model.generate(im.to(device), temperature=args.get('temperature', .2))
        pred = token2str4eval(detokenize(dec, dataset.tokenizer))
        pred = [list(map(str, dataset.tokenizer(pred_item)["input_ids"])) for pred_item in pred]
        truth = token2str4eval(detokenize(seq['input_ids'], dataset.tokenizer))
        truth = [list(map(str, dataset.tokenizer(truth_item)["input_ids"])) for truth_item in truth]
        bleus.append(metrics.bleu_score(pred, [alternatives(x) for x in truth]))

        for predi, truthi in zip(token2str(dec, dataset.tokenizer), token2str(seq['input_ids'], dataset.tokenizer)):
            ts = post_process4eval(truthi)
            if len(ts) > 0:
                edit_dists.append(distance(post_process4eval(predi), ts)/len(ts))
        
        dec = dec.cpu()
        tgt_seq = seq['input_ids'][:, 1:]
        shape_diff = dec.shape[1]-tgt_seq.shape[1]
        if shape_diff < 0:
            dec = torch.nn.functional.pad(dec, (0, -shape_diff), "constant", args.pad_token)
        elif shape_diff > 0:
            tgt_seq = torch.nn.functional.pad(tgt_seq, (0, shape_diff), "constant", args.pad_token)
        mask = torch.logical_or(tgt_seq != args.pad_token, dec != args.pad_token)
        tok_acc = (dec == tgt_seq)[mask].float().mean().item()
        token_acc.append(tok_acc)
        pbar.set_description('BLEU: %.3f, ED: %.2e, ACC: %.3f' % (np.mean(bleus), np.mean(edit_dists), np.mean(token_acc)))
        if num_batches is not None and i >= num_batches:
            break
    if len(bleus) > 0:
        bleu_score = np.mean(bleus)
        log[name+'/bleu'] = bleu_score
    if len(edit_dists) > 0:
        edit_distance = np.mean(edit_dists)
        log[name+'/edit_distance'] = edit_distance
    if len(token_acc) > 0:
        token_accuracy = np.mean(token_acc)
        log[name+'/token_acc'] = token_accuracy
    if args.wandb:
        # samples
        pred = token2str(dec, dataset.tokenizer)
        truth = token2str(seq['input_ids'], dataset.tokenizer)
        table = wandb.Table(columns=["Truth", "Prediction"])
        for k in range(min([len(pred), args.test_samples])):
            table.add_data(post_process4eval(truth[k]), post_process4eval(pred[k]))
        log[name+'/examples'] = table
        wandb.log(log)
    else:
        print('\n%s\n%s' % (truth, pred))
        print('BLEU: %.2f WER: %.2f Token_accuracy: %.2f' % (bleu_score, edit_distance, token_accuracy))
    return bleu_score, edit_distance, token_accuracy


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Test model')
    parser.add_argument('--config', default=None, help='path to yaml config file', type=str)
    parser.add_argument('-c', '--checkpoint', default=None, type=str, help='path to model checkpoint')
    parser.add_argument('-d', '--data', default='dataset/data/val.pkl', type=str, help='Path to Dataset pkl file')
    parser.add_argument('--no-cuda', action='store_true', help='Use CPU')
    parser.add_argument('-b', '--batchsize', type=int, default=10, help='Batch size')
    parser.add_argument('--debug', action='store_true', help='DEBUG')
    parser.add_argument('-t', '--temperature', type=float, default=.333, help='sampling emperature')
    parser.add_argument('-n', '--num-batches', type=int, default=None, help='how many batches to evaluate on. Defaults to None (all)')

    parsed_args = parser.parse_args()
    if parsed_args.config is None:
        with in_model_path():
            parsed_args.config = os.path.realpath('settings/config.yaml')
    with open(parsed_args.config, 'r') as f:
        params = yaml.load(f, Loader=yaml.FullLoader)
    args = parse_args(Munch(params))
    args.testbatchsize = parsed_args.batchsize
    args.wandb = False
    args.temperature = parsed_args.temperature
    logging.getLogger().setLevel(logging.DEBUG if parsed_args.debug else logging.WARNING)
    seed_everything(args.seed if 'seed' in args else 42)
    model = get_model(args)
    if parsed_args.checkpoint is None:
        with in_model_path():
            parsed_args.checkpoint = os.path.realpath('checkpoints/weights.pth')
    model.load_state_dict(torch.load(parsed_args.checkpoint, args.device))
    dataset = Im2LatexDataset().load(parsed_args.data)
    valargs = args.copy()
    valargs.update(batchsize=args.testbatchsize, keep_smaller_batches=True, test=True)
    dataset.update(**valargs)
    evaluate(model, dataset, args, num_batches=parsed_args.num_batches)


Overwriting /usr/local/lib/python3.8/dist-packages/pix2tex/eval.py


#edit file utils

In [35]:
# from google.colab import files
# files.view('/usr/local/lib/python3.7/dist-packages/pix2tex/utils/utils.py')

In [36]:
%%writefile /usr/local/lib/python3.8/dist-packages/pix2tex/utils/utils.py
import random
import os
import cv2
import re
from PIL import Image
import numpy as np
import torch
from munch import Munch
from inspect import isfunction
import contextlib

operators = '|'.join(['arccos', 'arcsin', 'arctan', 'arg', 'cos', 'cosh', 'cot', 'coth', 'csc', 'deg', 'det', 'dim', 'exp', 'gcd', 'hom', 'inf',
                      'injlim', 'ker', 'lg', 'lim', 'liminf', 'limsup', 'ln', 'log', 'max', 'min', 'Pr', 'projlim', 'sec', 'sin', 'sinh', 'sup', 'tan', 'tanh'])
ops = re.compile(r'\\operatorname{(%s)}' % operators)


class EmptyStepper:
    def __init__(self, *args, **kwargs):
        pass

    def step(self, *args, **kwargs):
        pass

# helper functions from lucidrains


def exists(val):
    return val is not None


def default(val, d):
    if exists(val):
        return val
    return d() if isfunction(d) else d


def seed_everything(seed: int):
    """Seed all RNGs

    Args:
        seed (int): seed
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


def parse_args(args, **kwargs) -> Munch:
    args = Munch({'epoch': 0}, **args)
    kwargs = Munch({'no_cuda': False, 'debug': False}, **kwargs)
    args.update(kwargs)
    args.wandb = not kwargs.debug and not args.debug
    args.device = get_device(args, kwargs.no_cuda)
    args.encoder_structure = args.get('encoder_structure', 'hybrid')
    args.max_dimensions = [args.max_width, args.max_height]
    args.min_dimensions = [args.get('min_width', 32), args.get('min_height', 32)]
    if 'decoder_args' not in args or args.decoder_args is None:
        args.decoder_args = {}
    return args


def get_device(args, no_cuda=False):
    device = 'cpu'
    available_gpus = torch.cuda.device_count()
    args.gpu_devices = args.gpu_devices if args.get('gpu_devices', False) else list(range(available_gpus))
    if available_gpus > 0 and not no_cuda:
        device = 'cuda:%d' % args.gpu_devices[0] if args.gpu_devices else 0
        assert available_gpus >= len(args.gpu_devices), "Available %d gpu, but specified gpu %s." % (available_gpus, ','.join(map(str, args.gpu_devices)))
        assert max(args.gpu_devices) < available_gpus, "legal gpu_devices should in [%s], received [%s]" % (','.join(map(str, range(available_gpus))), ','.join(map(str, args.gpu_devices)))
    return device


def gpu_memory_check(model, args):
    # check if largest batch can be handled by system
    try:
        batchsize = args.batchsize if args.get('micro_batchsize', -1) == -1 else args.micro_batchsize
        for _ in range(5):
            im = torch.empty(batchsize, args.channels, args.max_height, args.min_height, device=args.device).float()
            seq = torch.randint(0, args.num_tokens, (batchsize, args.max_seq_len), device=args.device).long()
            loss = model.data_parallel(im, device_ids=args.gpu_devices, tgt_seq=seq)
            loss.sum().backward()
    except RuntimeError:
        raise RuntimeError("The system cannot handle a batch size of %i for the maximum image size (%i, %i). Try to use a smaller micro batchsize." % (batchsize, args.max_height, args.max_width))
    model.zero_grad()
    with torch.cuda.device(args.device):
        torch.cuda.empty_cache()
    del im, seq

def token2str4eval(tokens):
    "token ở dạng list"
    tokens = [post_process4eval("".join(token)) for token in tokens]
    return tokens



def token2str(tokens, tokenizer) -> list:
    if len(tokens.shape) == 1:
        tokens = tokens[None, :]
    dec = [tokenizer.decode(tok) for tok in tokens]
    return [''.join(detok.split(' ')).replace('Ġ', ' ').replace('[EOS]', '').replace('[BOS]', '').replace('[PAD]', '').strip() for detok in dec]


def pad(img: Image, divable: int = 32) -> Image:
    """Pad an Image to the next full divisible value of `divable`. Also normalizes the image and invert if needed.

    Args:
        img (PIL.Image): input image
        divable (int, optional): . Defaults to 32.

    Returns:
        PIL.Image
    """
    threshold = 128
    data = np.array(img.convert('LA'))
    if data[..., -1].var() == 0:
        data = (data[..., 0]).astype(np.uint8)
    else:
        data = (255-data[..., -1]).astype(np.uint8)
    data = (data-data.min())/(data.max()-data.min())*255
    if data.mean() > threshold:
        # To invert the text to white
        gray = 255*(data < threshold).astype(np.uint8)
    else:
        gray = 255*(data > threshold).astype(np.uint8)
        data = 255-data

    coords = cv2.findNonZero(gray)  # Find all non-zero points (text)
    a, b, w, h = cv2.boundingRect(coords)  # Find minimum spanning bounding box
    rect = data[b:b+h, a:a+w]
    im = Image.fromarray(rect).convert('L')
    dims = []
    for x in [w, h]:
        div, mod = divmod(x, divable)
        dims.append(divable*(div + (1 if mod > 0 else 0)))
    padded = Image.new('L', dims, 255)
    padded.paste(im, (0, 0, im.size[0], im.size[1]))
    return padded


def remove1(str):
    """
    xóa bỏ các dấu \ bị thừa
    C.\ \ m>1.   --> C.m>1.
    """
    pattern = r"\\\s"
    repl = " "
    str = re.sub(pattern, repl, str)
    return str

def remove2(str):
    """
    Xóa bỏ left right 
    B.\frac{1}{\left(x^{2}-x\right).ln3}. --> B.\frac{1}{(x^{2}-x).ln3}.
    """
    pattern = r"(\\left)(.+?)(\\right)(.)"
    repl = r"\2\4"
    str = re.sub(pattern, repl, str)
    return str

def remove3(str):
    """ 
    thay \mid bằng |
    """
    pattern = r"\\mid"
    repl = r"|"
    str = re.sub(pattern, repl, str)
    return str

def remove4(str):
    """
    thay \circ bằng o vd 60^{\circ} bằng 60^{o}
    """
    pattern = r"(\{)(\\circ)(\})"
    repl = r"\1o\3"
    str = re.sub(pattern, repl, str)
    return str

def remove5(str):
    """
    xoa dau \ o cuoi cong thuc
    """
    pattern = r"(\\)$"
    repl = ""
    str = re.sub(pattern, repl, str)
    return str

def remove6(str):
    """
    xoa dau . o cuoi cong thuc
    """
    pattern = r"\.$"
    repl = ""
    str = re.sub(pattern, repl, str)
    return str

def remove8(str):
    """thay đổi \lbrack và rbrack thành [, ].

    Args:
        m\in\lbrack-4;0\rbrack
    """
    pattern = r"\\lbrack"
    repl = "["
    str = re.sub(pattern, repl, str)
    pattern = r"\\rbrack"
    repl = "]"
    str = re.sub(pattern, repl, str)
    return str

def remove9(str):
    """xóa bỏ \quad

    Args:
        d_{1}:\frac{x-4}{1}=\frac{y+2}{4}=\frac{z-1}{-2},\quad d_{_{2}}:\frac{x-2}{1}=\frac{y+1}{-1}=\frac{z-1}{1}.
    """
    pattern = r"\\quad"
    repl = ""
    str = re.sub(pattern, repl, str)
    return str

def remove10(str):
    """chuẩn hóa lại \cos - cos, \ln -->ln , \log -->log

    Args:
        str (_type_): _description_
    """
    pattern = r"\\cos"
    repl = "cos"
    str = re.sub(pattern, repl, str)
    pattern = r"\\sin"
    repl = "sin"
    str = re.sub(pattern, repl, str)
    pattern = r"\\ln"
    repl = "ln"
    str = re.sub(pattern, repl, str)
    pattern = r"\\log"
    repl = "log"
    str = re.sub(pattern, repl, str)
    return str

def remove11(str):
    """chuẩn hóa {0} thành {o}

    Args:
        str (_type_): _description_
    """
    pattern = r"(\^\{)(o)(\})"
    repl = r"\1 0\3"
    str = re.sub(pattern, repl, str)
    return str

def remove12(str):
    """thay => bằng \Rightarrow

    Args:
        str (_type_): _description_
    """
    pattern = r"=>"
    repl = r"\\Rightarrow "
    str = re.sub(pattern, repl, str)
    return str

def remove_whitespace(s):
    """Remove unnecessary whitespace from LaTeX code.
    """
    text_reg = r'(\\(operatorname|mathrm|text|mathbf)\s?\*? {.*?})'
    letter = '[a-zA-Z]'
    noletter = '[\W_^\d]'
    names = [x[0].replace(' ', '') for x in re.findall(text_reg, s)]
    s = re.sub(text_reg, lambda match: str(names.pop(0)), s)
    news = s
    while True:
        s = news
        news = re.sub(r'(?!\\ )(%s)\s+?(%s)' % (noletter, noletter), r'\1\2', s)
        news = re.sub(r'(?!\\ )(%s)\s+?(%s)' % (noletter, letter), r'\1\2', news)
        news = re.sub(r'(%s)\s+?(%s)' % (letter, noletter), r'\1\2', news)
        if news == s:
            break
    return s

def norm_data(str):
    """ 
    có 2 kiểu pattern:
    + có kí tự đặc biệt (không có dấu cách)
    + không có kí tự đặc biệt (Có dấu cách)
    """
    # print("input:", str)
    pattern_without_whitespace = r"\\[a-zA-Z]+"
    get_sp_pattern_withoutspace = "(" + "|".join(list(set(re.findall(pattern_without_whitespace, str)))) + ")"
    get_sp_pattern_withoutspace = get_sp_pattern_withoutspace.replace("\\","\\\\")

    str = re.sub(r"(?!\\ )([\W_^\d])\s+([a-zA-Z1-9])",r"\1\2",str)
    str = re.sub(r"(?!\\ )([a-zA-Z1-9])\s+([\W_^\d])",r"\1\2",str)
    pattern_with_whitespace = r"\\[a-zA-Z]+\s"
    get_sp_pattern_withspace = "(" + "|".join(list(set(re.findall(pattern_with_whitespace, str)))) + ")"
    get_sp_pattern_withspace = get_sp_pattern_withspace.replace("\\","\\\\").replace(" ","")
    # print("pattern with space",get_sp_pattern_withspace)
    str = re.sub(r"\\\s",r"TranNgocDU",str)
    str = re.sub(r"\s",r"",str)
    # str = re.sub(r"(?!\\ )([a-zA-Z])\s+([a-zA-Z])",r"\1\2",str)
    # str = re.sub(r"(?!\\ )([\W_^\d])\s+([a-zA-Z1-9])",r"\1\2",str)
    # str = re.sub(r"(?!\\ )([a-zA-Z1-9])\s+([\W_^\d])",r"\1\2",str)
    # str = re.sub(r"(?!\\ )([\W_^\d])\s+([\W_^\d])",r"\1\2",str)
    # print("str:",str)
    # print("pattern without space:",get_sp_pattern_withoutspace)
    if len(get_sp_pattern_withspace) > 2:
        str = re.sub(get_sp_pattern_withoutspace,r"\1",str)
    # print(str)
    if len(get_sp_pattern_withspace) > 2:
        str = re.sub(get_sp_pattern_withspace, r"\1 ", str)
    str = re.sub(r"TranNgocDU",r"\\ ", str)
    return str

def post_process4eval(str):
    tem = str
    while True:
        str = remove1(str)  #xóa bỏ các dấu \ bị thừa
        str = remove2(str)  #Xóa bỏ left right 
        str = remove3(str)  #thay \mid bằng |
        str = remove4(str)  #thay \circ bằng o vd 60^{\circ} bằng 60^{o}
        str = remove5(str)  #xoa dau \ o cuoi cong thuc
        str = remove6(str)  #xoa dau . o cuoi cong thuc
        str = remove8(str)  #thay đổi \lbrack và rbrack thành [, ].
        str = remove9(str)  #xóa bỏ \quad
        str = remove10(str) #chuẩn hóa lại \cos - cos, \ln -->ln , \log -->log
        str = remove11(str) #chuẩn hóa lại {o} thành {0}
        str = remove12(str) #thay => bằng \Rightarrow
        str = norm_data(str)
        if str == tem:
            break
        tem = str
    return str

def post_process(s: str):
    """Remove unnecessary whitespace from LaTeX code.

    Args:
        s (str): Input string

    Returns:
        str: Processed image
    """
    text_reg = r'(\\(operatorname|mathrm|text|mathbf)\s?\*? {.*?})'
    letter = '[a-zA-Z]'
    noletter = '[\W_^\d]'
    names = [x[0].replace(' ', '') for x in re.findall(text_reg, s)]
    s = re.sub(text_reg, lambda match: str(names.pop(0)), s)
    news = s
    while True:
        s = news
        news = re.sub(r'(?!\\ )(%s)\s+?(%s)' % (noletter, noletter), r'\1\2', s)
        news = re.sub(r'(?!\\ )(%s)\s+?(%s)' % (noletter, letter), r'\1\2', news)
        news = re.sub(r'(%s)\s+?(%s)' % (letter, noletter), r'\1\2', news)
        if news == s:
            break
    return s



def alternatives(s):
    # TODO takes list of list of tokens
    # try to generate equivalent code eg \ne \neq or \to \rightarrow
    # alts = [s]
    # names = ['\\'+x for x in re.findall(ops, s)]
    # alts.append(re.sub(ops, lambda match: str(names.pop(0)), s))

    # return alts
    return [s]


def get_optimizer(optimizer):
    return getattr(torch.optim, optimizer)


def get_scheduler(scheduler):
    if scheduler is None:
        return EmptyStepper
    return getattr(torch.optim.lr_scheduler, scheduler)


def num_model_params(model):
    return sum([p.numel() for p in model.parameters()])


@contextlib.contextmanager
def in_model_path():
    import pix2tex
    model_path = os.path.join(os.path.dirname(pix2tex.__file__), 'model')
    saved = os.getcwd()
    os.chdir(model_path)
    try:
        yield
    finally:
        os.chdir(saved)


Overwriting /usr/local/lib/python3.8/dist-packages/pix2tex/utils/utils.py


# edit file train

In [33]:
# from google.colab import files
# files.view('/usr/local/lib/python3.7/dist-packages/pix2tex/train.py')

In [97]:
%%writefile /usr/local/lib/python3.8/dist-packages/pix2tex/train.py
from pix2tex.dataset.dataset import Im2LatexDataset
import os
import argparse
import logging
import yaml

import torch
from munch import Munch
from tqdm.auto import tqdm
import wandb
import torch.nn as nn
from pix2tex.eval import evaluate
from pix2tex.models import get_model
# from pix2tex.utils import *
from pix2tex.utils import in_model_path, parse_args, seed_everything, get_optimizer, get_scheduler, gpu_memory_check



def train(args):
    dataloader = Im2LatexDataset().load(args.data)
    dataloader.update(**args, test=False)
    valdataloader = Im2LatexDataset().load(args.valdata)
    valargs = args.copy()
    valargs.update(batchsize=args.testbatchsize, keep_smaller_batches=True, test=True)
    valdataloader.update(**valargs)
    device = args.device
    model = get_model(args)
    if torch.cuda.is_available() and not args.no_cuda:
        gpu_memory_check(model, args)
    max_bleu, max_token_acc = 0, 0
    out_path = os.path.join(args.model_path, args.name)
    os.makedirs(out_path, exist_ok=True)

    if args.load_chkpt is not None:
        model.load_state_dict(torch.load(args.load_chkpt, map_location=device))

    def save_models(e, step=0):
        torch.save(model.state_dict(), os.path.join("/content/drive/MyDrive/", '%s_e%02d_step%02d.pth' % (args.name, e+1, step)))
        yaml.dump(dict(args), open(os.path.join("/content/drive/MyDrive/", 'config.yaml'), 'w+'))

    opt = get_optimizer(args.optimizer)(model.parameters(), args.lr, betas=args.betas)
    scheduler = get_scheduler(args.scheduler)(opt, step_size=args.lr_step, gamma=args.gamma)

    microbatch = args.get('micro_batchsize', -1)
    if microbatch == -1:
        microbatch = args.batchsize

    try:
        for e in range(args.epoch, args.epochs):
            args.epoch = e
            dset = tqdm(iter(dataloader))
            for i, (seq, im, _) in enumerate(dset):
                if seq is not None and im is not None:
                    opt.zero_grad()
                    total_loss = 0
                    for j in range(0, len(im), microbatch):
                        tgt_seq, tgt_mask = seq['input_ids'][j:j+microbatch].to(device), seq['attention_mask'][j:j+microbatch].bool().to(device)
                        loss = model.data_parallel(im[j:j+microbatch].to(device), device_ids=args.gpu_devices, tgt_seq=tgt_seq, mask=tgt_mask)*microbatch/args.batchsize
                        loss.backward()  # data parallism loss is a vector
                        total_loss += loss.item()
                        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
                    opt.step()
                    scheduler.step()
                    dset.set_description('Loss: %.4f' % total_loss)
                    if args.wandb:
                        wandb.log({'train/loss': total_loss})
            
            bleu_score, edit_distance, token_accuracy = evaluate(model, valdataloader, args, num_batches=None, name='val')
            if bleu_score > max_bleu and token_accuracy > max_token_acc:
                max_bleu, max_token_acc = bleu_score, token_accuracy
                save_models(e, step=10000)
            if (e+1) % args.save_freq == 0:
                save_models(e, step=len(dataloader))
            if args.wandb:
                wandb.log({'train/epoch': e+1})
    except KeyboardInterrupt:
        if e >= 2:
            save_models(e, step=i)
        raise KeyboardInterrupt
    save_models(e, step=len(dataloader))


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Train model')
    parser.add_argument('--config', default=None, help='path to yaml config file', type=str)
    parser.add_argument('--no_cuda', action='store_true', help='Use CPU')
    parser.add_argument('--debug', action='store_true', help='DEBUG')
    parser.add_argument('--resume', help='path to checkpoint folder', action='store_true')
    parsed_args = parser.parse_args()
    if parsed_args.config is None:
        with in_model_path():
            parsed_args.config = os.path.realpath('settings/debug.yaml')
    with open(parsed_args.config, 'r') as f:
        params = yaml.load(f, Loader=yaml.FullLoader)
    args = parse_args(Munch(params), **vars(parsed_args))
    logging.getLogger().setLevel(logging.DEBUG if parsed_args.debug else logging.WARNING)
    seed_everything(args.seed)
    if args.wandb:
        if not parsed_args.resume:
            args.id = wandb.util.generate_id()
        wandb.init(config=dict(args), resume='allow', name=args.name, id=args.id)
        args = Munch(wandb.config)
    train(args)


Overwriting /usr/local/lib/python3.8/dist-packages/pix2tex/train.py


# make dataset

In [37]:
from sklearn.model_selection import train_test_split

In [38]:
import pandas as pd
fr = pd.read_csv("/content/data_real.csv")
fr = fr.dropna()
fr.to_csv("/content/data_real.csv")

In [78]:
handwrite = pd.read_csv("/content/HandwriteFormula/handwritting.csv")
images = []
formulas = []
for path, formula in zip(handwrite["image"], handwrite["formula"]):
  try:
    # print("/content/HandwriteFormula/CROHME/"+path)
    img_test = Image.open("/content/HandwriteFormula2/"+path)
    images.append(path)
    formulas.append(formula)
  except:
    pass

dataFrame = pd.DataFrame({"image":images, "formula":formulas}).to_csv("/content/HandwriteFormula/handwritting1.csv", index=False)

In [79]:
handwrite = pd.read_csv("/content/HandwriteFormula/handwritting1.csv")
handwrite = handwrite.dropna()
train_csv, test_csv = train_test_split(handwrite,test_size=1000)

In [80]:
train_csv.to_csv("train_handwrite.csv", index=False)
test_csv.to_csv("test_handwrite.csv", index=False)

In [64]:
Aida_data = pd.read_csv("/content/Aida_data.csv")
Aida_data = Aida_data.dropna()
train_csv, test_csv = train_test_split(Aida_data,test_size=1000)
train_csv.to_csv("Aida_train_handwrite.csv", index=False)
test_csv.to_csv("Aida_test_handwrite.csv", index=False)

In [82]:
!python -m pix2tex.dataset.dataset -i /content/datareal/1110/ /content/HandwriteFormula2/ /content/AidaImage/ -e /content/data_real.csv /content/train_handwrite.csv /content/Aida_train_handwrite.csv -o /content/train.pkl

Generate dataset
start make file dataset.pkl
make dataset
100% 11216/11216 [00:00<00:00, 48743.13it/s]
success!
start make file dataset.pkl
make dataset
100% 9821/9821 [00:00<00:00, 56369.14it/s]
success!
start make file dataset.pkl
make dataset
100% 99000/99000 [00:15<00:00, 6526.24it/s]
success!


In [66]:
# !python -m pix2tex.dataset.dataset -i /content/data_train/data_train/ /content/data_add/data_add/ /content/datareal/1110/ /content/HandwriteFormula2/CROHME/ -e /content/train_data_core_norm.csv /content/train_data_core_norm_background.csv /content/data_real.csv /content/train_handwrite.csv -o /content/train.pkl

In [ ]:
# !python -m pix2tex.dataset.dataset -i /content/datareal/1110/ -e /content/data_real.csv -o /content/train.pkl

In [ ]:
# !python -m pix2tex.dataset.dataset -i /content/data_train/data_train/   -e /content/train_data_core_norm.csv -o /content/train.pkl

In [89]:
!python -m pix2tex.dataset.dataset -i /content/images_private_test/images_private_test/ -e /content/private_test_norm.csv -o /content/val_on.pkl

Generate dataset
start make file dataset.pkl
make dataset
100% 1000/1000 [00:00<00:00, 53518.57it/s]
success!


In [84]:
!python -m pix2tex.dataset.dataset -i /content/HandwriteFormula2/ -e /content/test_handwrite.csv -o /content/test_handwrite.pkl

Generate dataset
start make file dataset.pkl
make dataset
100% 1000/1000 [00:00<00:00, 54143.11it/s]
success!


In [85]:
!python -m pix2tex.dataset.dataset -i /content/AidaImage/ -e /content/Aida_test_handwrite.csv -o /content/Aida_test_handwrite.pkl

Generate dataset
start make file dataset.pkl
make dataset
100% 1000/1000 [00:00<00:00, 10327.21it/s]
success!


In [90]:
!python -m pix2tex.dataset.dataset -i /content/AidaImage/ /content/HandwriteFormula2/ -e /content/Aida_test_handwrite.csv /content/test_handwrite.csv -o /content/val_off.pkl

Generate dataset
start make file dataset.pkl
make dataset
100% 1000/1000 [00:00<00:00, 46232.49it/s]
success!
start make file dataset.pkl
make dataset
100% 1000/1000 [00:00<00:00, 47151.38it/s]
success!


In [91]:
# generate colab specific config (set 'debug' to true if wandb is not used)
!echo {backbone_layers: [2, 3, 7], betas: [0.9, 0.999], batchsize: 32, bos_token: 1, channels: 1, data: /content/train.pkl, debug: true, decoder_args: {'attn_on_attn': true, 'cross_attend': true, 'ff_glu': true, 'rel_pos_bias': false, 'use_scalenorm': false}, dim: 256, encoder_depth: 4, eos_token: 2, epochs: 50, gamma: 0.9995, heads: 8, id: null, load_chkpt: '/content/drive/MyDrive/mixed_e03_step6267_1.47e-1.pth', lr: 0.001, lr_step: 2, max_height: 192, max_seq_len: 512, max_width: 672, min_height: 32, min_width: 32, model_path: checkpoints, name: mixed, num_layers: 4, num_tokens: 8000, optimizer: Adam, output_path: outputs, pad: false, pad_token: 0, patch_size: 16, sample_freq: 2000, save_freq: 1, scheduler: StepLR, seed: 42, temperature: 0.2, test_samples: 5, testbatchsize: 16, tokenizer: dataset/tokenizer.json, valbatches: 100, valdata: /content/val_off.pkl} > colab.yaml

# train

In [ ]:
!pip uninstall wandb -y
!pip install wandb

In [99]:
!python -m pix2tex.train --config colab.yaml

Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.8/dist-packages/pix2tex/train.py", line 100, in <module>
    args.id = wandb.util.generate_id()
AttributeError: module 'wandb.util' has no attribute 'generate_id'


In [46]:
!python -m pix2tex.eval -c '/content/drive/MyDrive/mixed_e03_step6267_1.47e-1.pth' -d /content/Aida_test_handwrite.pkl

start make file dataset.pkl
BLEU: 0.113, ED: 8.64e-01, ACC: 0.075: 100% 118/118 [01:43<00:00,  1.14it/s]

[['61', '747', '64', '92', '61', '219', '61', '249', '14', '61', '279', '94', '61', '117', '92', '61', '413', '92', '61', '219', '94', '94', '92', '61', '219', '63', '92', '20', '94', '94'], ['61', '747', '64', '92', '67', '61', '249', '61', '117', '92', '61', '171', '94', '92', '26', '94', '94', '314', '63', '92', '21', '94', '92', '67', '94', '12', '61', '603', '63', '92', '19', '94', '92', '67', '94'], ['61', '747', '64', '92', '89', '61', '249', '18', '94', '61', '117', '92', '24', '295', '92', '9', '22', '89', '10', '94', '94', '92', '23', '89', '295', '92', '9', '89', '10', '94', '94'], ['61', '747', '64', '92', '81', '61', '249', '61', '117', '92', '61', '171', '94', '92', '25', '94', '94', '61', '117', '92', '314', '92', '81', '94', '12', '14', '19', '61', '413', '92', '81', '94', '94', '92', '81', '12', '14', '26', '61', '117', '92', '61', '171', '94', '92', '23', '94', '9

In [ ]:
!python -m pix2tex.eval -c '/content/drive/MyDrive/mixed_e08_step5587.pth' -d /content/val.pkl

start make file dataset.pkl
BLEU: 0.785, ED: 1.80e-01, ACC: 0.607: 100% 124/124 [01:02<00:00,  1.98it/s]

[['36', '64', '92', '19', '94', '41', '64', '92', '22', '94', '48', '41'], ['34', '77', '64', '92', '21', '94', '36', '64', '92', '20', '94'], ['61', '563', '17', '28', '19', '61', '611'], ['37', '15', '61', '9', '61', '61', '117', '92', '21', '94', '92', '20', '94', '28', '19', '10', '61'], ['36', '15', '61', '89', '30', '21'], ['34', '15', '61', '89', '30', '14', '18'], ['36', '30', '19', '36', '64', '92', '17', '94'], ['34', '15', '61', '78', '30', '17'], ['47', '30', '22', '17', '17', '17'], ['36', '15', '61', '78', '31', '17']]
[['36', '64', '92', '19', '94', '41', '64', '92', '22', '94', '48', '41'], ['34', '77', '64', '92', '21', '94', '36', '64', '92', '20', '94'], ['60', '17', '28', '19', '62'], ['37', '15', '61', '9', '61', '117', '92', '21', '94', '92', '20', '94', '28', '19', '10'], ['36', '15', '61', '89', '30', '21'], ['34', '15', '61', '89', '30', '14', '18'], ['36',

In [ ]:
!python -m pix2tex.eval -c '/content/drive/MyDrive/mixed_e09_step5587.pth' -d /content/val.pkl

start make file dataset.pkl
BLEU: 0.783, ED: 1.76e-01, ACC: 0.600: 100% 124/124 [00:58<00:00,  2.12it/s]

[['36', '64', '92', '19', '94', '41', '64', '92', '22', '94', '48', '41'], ['34', '77', '64', '92', '21', '94', '36', '64', '92', '20', '94'], ['61', '563', '17', '28', '19', '61', '611'], ['37', '15', '61', '9', '61', '61', '117', '92', '21', '94', '92', '20', '94', '28', '19', '10', '61'], ['36', '15', '61', '89', '30', '21'], ['34', '15', '61', '89', '30', '14', '18'], ['36', '30', '19', '36', '64', '92', '17', '94'], ['34', '15', '61', '78', '30', '17'], ['47', '30', '22', '17', '17', '17'], ['36', '15', '61', '78', '31', '17']]
[['36', '64', '92', '19', '94', '41', '64', '92', '22', '94', '48', '41'], ['34', '77', '64', '92', '21', '94', '36', '64', '92', '20', '94'], ['60', '17', '28', '19', '62'], ['37', '15', '61', '9', '61', '117', '92', '21', '94', '92', '20', '94', '28', '19', '10'], ['36', '15', '61', '89', '30', '21'], ['34', '15', '61', '89', '30', '14', '18'], ['36',

In [ ]:
!python -m pix2tex.eval -c '/content/drive/MyDrive/mixed_e01_step5587.pth' -d /content/val.pkl

start make file dataset.pkl
BLEU: 0.778, ED: 1.89e-01, ACC: 0.595: 100% 124/124 [01:01<00:00,  2.00it/s]

[['36', '64', '92', '19', '94', '41', '64', '92', '22', '94', '48', '41'], ['34', '77', '64', '92', '21', '94', '36', '64', '92', '20', '94'], ['61', '563', '17', '28', '19', '61', '611'], ['37', '15', '61', '9', '61', '61', '117', '92', '21', '94', '92', '20', '94', '28', '19', '10', '61'], ['36', '15', '61', '89', '30', '21'], ['34', '15', '61', '89', '30', '14', '18'], ['36', '30', '19', '36', '64', '92', '17', '94'], ['34', '15', '61', '78', '30', '17'], ['47', '30', '22', '17', '17', '17'], ['36', '15', '61', '78', '31', '17']]
[['36', '64', '92', '19', '94', '41', '64', '92', '22', '94', '48', '41'], ['34', '77', '64', '92', '21', '94', '36', '64', '92', '20', '94'], ['61', '563', '17', '28', '19', '61', '611'], ['37', '15', '61', '9', '61', '117', '92', '21', '94', '92', '20', '94', '89', '10'], ['36', '15', '61', '89', '30', '21'], ['34', '15', '61', '89', '30', '14', '18']

In [ ]:
!python -m pix2tex.eval -c /content/drive/MyDrive/mixed_e11_step4890.pth -d /content/val.pkl

In [ ]:
%ls /usr/local/lib/python3.7/dist-packages/pix2tex/model/dataset

#edit file cli

In [ ]:
%%writefile /usr/local/lib/python3.7/dist-packages/pix2tex/cli.py
from pix2tex.dataset.transforms import test_transform
import pandas.io.clipboard as clipboard
from PIL import ImageGrab
from PIL import Image
import os
from pathlib import Path
import sys
from typing import List, Optional, Tuple
import atexit
from contextlib import suppress
import logging
import yaml
import re

with suppress(ImportError):
    import readline

import numpy as np
import torch
from torch._appdirs import user_data_dir
from munch import Munch
from transformers import PreTrainedTokenizerFast
from timm.models.resnetv2 import ResNetV2
from timm.models.layers import StdConv2dSame

from pix2tex.dataset.latex2png import tex2pil
from pix2tex.models import get_model
from pix2tex.utils import *
from pix2tex.model.checkpoints.get_latest_checkpoint import download_checkpoints





class LatexOCR:
    '''Get a prediction of an image in the easiest way'''

    image_resizer = None
    last_pic = None

    @in_model_path()
    def __init__(self, arguments=None):
        """Initialize a LatexOCR model

        Args:
            arguments (Union[Namespace, Munch], optional): Special model parameters. Defaults to None.
        """
        if arguments is None:
            arguments = Munch({'config': 'settings/config.yaml', 'checkpoint': '/content/drive/MyDrive/mixed_e08_step5587.pth', 'no_cuda': False, 'no_resize': False})
        logging.getLogger().setLevel(logging.FATAL)
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
        with open(arguments.config, 'r') as f:
            params = yaml.load(f, Loader=yaml.FullLoader)
        self.args = parse_args(Munch(params))
        self.args.update(**vars(arguments))
        self.args.wandb = False
        self.args.device = 'cuda' if torch.cuda.is_available() and not self.args.no_cuda else 'cpu'
        if not os.path.exists(self.args.checkpoint):
            download_checkpoints()
        self.model = get_model(self.args)
        self.model.load_state_dict(torch.load(self.args.checkpoint, map_location=self.args.device))
        self.model.eval()
        self.tokenizer = PreTrainedTokenizerFast(tokenizer_file=self.args.tokenizer)

    @in_model_path()
    def __call__(self, img=None, resize=False) -> str:
        """Get a prediction from an image

        Args:
            img (Image, optional): Image to predict. Defaults to None.
            resize (bool, optional): Whether to call the resize model. Defaults to True.

        Returns:
            str: predicted Latex code
        """
    
        t = test_transform(image=img)['image'][:1].unsqueeze(0)
        im = t.to(self.args.device)

        dec = self.model.generate(im.to(self.args.device), temperature=self.args.get('temperature', .25))
        pred = post_process(token2str(dec, self.tokenizer)[0])
        try:
            clipboard.copy(pred)
        except:
            pass
        return pred


def output_prediction(pred, args):
    TERM = os.getenv('TERM', 'xterm')
    if not sys.stdout.isatty():
        TERM = 'dumb'
    try:
        from pygments import highlight
        from pygments.lexers import get_lexer_by_name
        from pygments.formatters import get_formatter_by_name

        if TERM.split('-')[-1] == '256color':
            formatter_name = 'terminal256'
        elif TERM != 'dumb':
            formatter_name = 'terminal'
        else:
            formatter_name = None
        if formatter_name:
            formatter = get_formatter_by_name(formatter_name)
            lexer = get_lexer_by_name('tex')
            print(highlight(pred, lexer, formatter), end='')
    except ImportError:
        TERM = 'dumb'
    if TERM == 'dumb':
        print(pred)
    if args.show or args.katex:
        try:
            if args.katex:
                raise ValueError
            tex2pil([f'$${pred}$$'])[0].show()
        except Exception as e:
            # render using katex
            import webbrowser
            from urllib.parse import quote
            url = 'https://katex.org/?data=' + \
                quote('{"displayMode":true,"leqno":false,"fleqn":false,"throwOnError":true,"errorColor":"#cc0000",\
"strict":"warn","output":"htmlAndMathml","trust":false,"code":"%s"}' % pred.replace('\\', '\\\\'))
            webbrowser.open(url)


def predict(model, file, arguments):
    img = None
    if file:
        try:
            img = Image.open(os.path.expanduser(file))
        except Exception as e:
            print(e, end='')
    else:
        try:
            img = ImageGrab.grabclipboard()
        except NotImplementedError as e:
            print(e, end='')
    pred = model(img)
    output_prediction(pred, arguments)

def check_file_path(paths:List[Path], wdir:Optional[Path]=None)->List[str]:
    files = []
    for path in paths:
        if type(path)==str:
            if path=='':
                continue
            path=Path(path)
        pathsi = ([path] if wdir is None else [path, wdir/path])
        for p in pathsi:
            if p.exists():
                files.append(str(p.resolve()))
            elif '*' in path.name:
                files.extend([str(pi.resolve()) for pi in p.parent.glob(p.name)])
    return list(set(files))

def main(arguments):
    path = user_data_dir('img2tex')
    os.makedirs(path, exist_ok=True)
    history_file = os.path.join(path, 'history.txt')
    with suppress(NameError):
        # user can `ln -s /dev/null ~/.local/share/pix2tex/history.txt` to
        # disable history record
        with suppress(OSError):
            readline.read_history_file(history_file)
        atexit.register(readline.write_history_file, history_file)
    files = check_file_path(arguments.file)
    wdir = Path(os.getcwd())
    with in_model_path():
        model = LatexOCR(arguments)
        if files:
            for file in check_file_path(arguments.file, wdir):
                print(file + ': ', end='')
                predict(model, file, arguments)
                model.last_pic = None
                with suppress(NameError):
                    readline.add_history(file)
            exit()
        pat = re.compile(r't=([\.\d]+)')
        while True:
            try:
                instructions = input('Predict LaTeX code for image ("h" for help). ')
            except KeyboardInterrupt:
                # TODO: make the last line gray
                print("")
                continue
            except EOFError:
                break
            file = instructions.strip()
            ins = file.lower()
            t = pat.match(ins)
            if ins == 'x':
                break
            elif ins in ['?', 'h', 'help']:
                print('''pix2tex help:

    Usage:
        On Windows and macOS you can copy the image into memory and just press ENTER to get a prediction.
        Alternatively you can paste the image file path here and submit.

        You might get a different prediction every time you submit the same image. If the result you got was close you
        can just predict the same image by pressing ENTER again. If that still does not work you can change the temperature
        or you have to take another picture with another resolution (e.g. zoom out and take a screenshot with lower resolution). 

        Press "x" to close the program.
        You can interrupt the model if it takes too long by pressing Ctrl+C.

    Visualization:
        You can either render the code into a png using XeLaTeX (see README) to get an image file back.
        This is slow and requires a working installation of XeLaTeX. To activate type 'show' or set the flag --show
        Alternatively you can render the expression in the browser using katex.org. Type 'katex' or set --katex

    Settings:
        to toggle one of these settings: 'show', 'katex', 'no_resize' just type it into the console
        Change the temperature (default=0.333) type: "t=0.XX" to set a new temperature.
                    ''')
                continue
            elif ins in ['show', 'katex', 'no_resize']:
                setattr(arguments, ins, not getattr(arguments, ins, False))
                print('set %s to %s' % (ins, getattr(arguments, ins)))
                continue
            elif t is not None:
                t = t.groups()[0]
                model.args.temperature = float(t)+1e-8
                print('new temperature: T=%.3f' % model.args.temperature)
                continue
            files = check_file_path(file.split(' '), wdir)
            with suppress(KeyboardInterrupt):
                if files:
                    for file in files:
                        if len(files)>1:
                            print(file + ': ', end='')
                        predict(model, file, arguments)
                else:
                    predict(model, file, arguments)
            file = None


Overwriting /usr/local/lib/python3.7/dist-packages/pix2tex/cli.py


# make them data

In [ ]:
%mkdir data_debug2

In [ ]:
!pip install torchmetrics

In [ ]:
from cv2 import blur
from torch import equal
from pix2tex import cli as img2tex
import cv2
from PIL import Image
import pandas as pd
import io
import matplotlib.pyplot as plt
from IPython.lib.latextools import latex_to_png
import matplotlib
from torchtext.data import metrics
import tqdm
import glob
from torchmetrics.functional import char_error_rate
from transformers import PreTrainedTokenizerFast
from PIL import ImageDraw 
from pix2tex.utils import *

import numpy as np

model = img2tex.LatexOCR()
tokenizer = PreTrainedTokenizerFast(tokenizer_file="/usr/local/lib/python3.7/dist-packages/pix2tex/model/dataset/tokenizer.json")



In [ ]:
%ls /usr/local/lib/python3.7/dist-packages/pix2tex/model/dataset/tokenizer.json

tokenizer.json


# make file.pkl cho viec tao data debug

In [ ]:
# %%writefile /usr/local/lib/python3.7/dist-packages/pix2tex/dataset/dataset.py
import torch
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import imagesize
import logging
import glob
import os
from os.path import join
from collections import defaultdict
import pickle
import cv2
from transformers import PreTrainedTokenizerFast
from tqdm.auto import tqdm

from pix2tex.utils.utils import in_model_path
from pix2tex.dataset.transforms import train_transform, test_transform


import pandas as pd


class Im2LatexDataset:
    keep_smaller_batches = True
    shuffle = False
    batchsize = 32
    max_dimensions = (1024, 512)
    min_dimensions = (32, 32)
    max_seq_len = 1024
    pad_token = "[PAD]"
    bos_token = "[BOS]"
    eos_token = "[EOS]"
    pad_token_id = 0
    bos_token_id = 1
    eos_token_id = 2
    transform = train_transform
    data = defaultdict(lambda: [])

    def __init__(self, equations="abc", images="/content/datareal/1110/*", tokenizer="/usr/local/lib/python3.7/dist-packages/pix2tex/model/dataset/tokenizer.json", shuffle=False, batchsize=32, max_seq_len=1024,
                 max_dimensions=(1024, 512), min_dimensions=(32, 32), pad=False, keep_smaller_batches=True, test=True):
     
        print("start make file dataset.pkl")
        if images is not None and equations is not None:
            assert tokenizer is not None
            self.images = [path for path in glob.glob(images)]
            self.sample_size = len(self.images)
            self.imgPath = images
            self.tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer)
            self.shuffle = shuffle
            self.batchsize = batchsize
            self.max_seq_len = max_seq_len
            self.max_dimensions = max_dimensions
            self.min_dimensions = min_dimensions
            self.pad = pad
            self.keep_smaller_batches = keep_smaller_batches
            self.test = test
            # check the image dimension for every image and group them together
            try:
                print("make dataset")
                for i, im in tqdm(enumerate(self.images), total=len(self.images)):
                    path = im
                    width, height = imagesize.get(path)
                    if min_dimensions[0] <= width <= max_dimensions[0] and min_dimensions[1] <= height <= max_dimensions[1]:
                        self.data[(width, height)].append(("ab c da", path))
                print("success!")
            except KeyboardInterrupt:
                pass
            self.data = dict(self.data)
            self._get_size()

            iter(self)

    def __len__(self):
        return self.size

    def __iter__(self):
        # dùng để chia batch
        self.i = 0
        self.transform = test_transform if self.test else train_transform
        self.pairs = []
        for k in self.data:
            info = np.array(self.data[k], dtype=object)
            p = torch.randperm(len(info)) if self.shuffle else torch.arange(len(info))
            for i in range(0, len(info), self.batchsize):
                batch = info[p[i:i+self.batchsize]]
                if len(batch.shape) == 1:
                    batch = batch[None, :]
                if len(batch) < self.batchsize and not self.keep_smaller_batches:
                    continue
                self.pairs.append(batch)
        if self.shuffle:
            self.pairs = np.random.permutation(np.array(self.pairs, dtype=object))
        else:
            self.pairs = np.array(self.pairs, dtype=object)
        self.size = len(self.pairs)
        return self

    def __next__(self):
        # dùng để load batch
        if self.i >= self.size:
            raise StopIteration
        self.i += 1
        return self.prepare_data(self.pairs[self.i-1])

    def prepare_data(self, batch):
        """loads images into memory

        Args:
            batch (numpy.array[[str, str]]): array of equations and image path pairs

        Returns:
            tuple(torch.tensor, torch.tensor): data in memory
        """

        eqs, ims = batch.T
        tok = self.tokenizer(list(eqs), return_token_type_ids=False)
        # pad with bos and eos token
        for k, p in zip(tok, [[self.bos_token_id, self.eos_token_id], [1, 1]]):
            tok[k] = pad_sequence([torch.LongTensor([p[0]]+x+[p[1]]) for x in tok[k]], batch_first=True, padding_value=self.pad_token_id)
        # check if sequence length is too long
        if self.max_seq_len < tok['attention_mask'].shape[1]:
            return next(self)
        images = []
        images_path = []
        for path in list(ims):
            im = cv2.imread(path)
            if im is None:
                print(path, 'not found!')
                continue
            im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            if not self.test:
                # sometimes convert to bitmask
                if np.random.random() < .04:
                    im[im != 255] = 0
            images.append(self.transform(image=im)['image'][:1])
            images_path.append(path)
        try:
            images = torch.cat(images).float().unsqueeze(1)
        except RuntimeError:
            logging.critical('Images not working: %s' % (' '.join(list(ims))))
            return None, None, None
        if self.pad:
            h, w = images.shape[2:]
            images = F.pad(images, (0, self.max_dimensions[0]-w, 0, self.max_dimensions[1]-h), value=1)
        return tok, images, images_path

    def _get_size(self):
        self.size = 0
        for k in self.data:
            div, mod = divmod(len(self.data[k]), self.batchsize)
            self.size += div  # + (1 if mod > 0 else 0)

    def load(self, filename, args=[]):
        """returns a pickled version of a dataset

        Args:
            filename (str): Path to dataset
        """
        if not os.path.exists(filename):
            with in_model_path():
                tempf = os.path.join('..', filename)
                if os.path.exists(tempf):
                    filename = os.path.realpath(tempf)
        with open(filename, 'rb') as file:
            x = pickle.load(file)
        return x

    def combine(self, x):
        """Combine Im2LatexDataset with another Im2LatexDataset

        Args:
            x (Im2LatexDataset): Dataset to absorb
        """
        for key in x.data.keys():
            if key in self.data.keys():
                self.data[key].extend(x.data[key])
                self.data[key] = list(set(self.data[key]))
            else:
                self.data[key] = x.data[key]
        self._get_size()
        iter(self)

    def save(self, filename):
        """save a pickled version of a dataset

        Args:
            filename (str): Path to dataset
        """
        with open(filename, 'wb') as file:
            pickle.dump(self, file)

    def update(self, **kwargs):
        for k in ['batchsize', 'shuffle', 'pad', 'keep_smaller_batches', 'test', 'max_seq_len']:
            if k in kwargs:
                setattr(self, k, kwargs[k])
        if 'max_dimensions' in kwargs or 'min_dimensions' in kwargs:
            if 'max_dimensions' in kwargs:
                self.max_dimensions = kwargs['max_dimensions']
            if 'min_dimensions' in kwargs:
                self.min_dimensions = kwargs['min_dimensions']
            temp = {}
            for k in self.data:
                if self.min_dimensions[0] <= k[0] <= self.max_dimensions[0] and self.min_dimensions[1] <= k[1] <= self.max_dimensions[1]:
                    temp[k] = self.data[k]
            self.data = temp
        if 'tokenizer' in kwargs:
            tokenizer_file = kwargs['tokenizer']
            if not os.path.exists(tokenizer_file):
                with in_model_path():
                    tokenizer_file = os.path.realpath(tokenizer_file)
            self.tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_file)
        self._get_size()
        iter(self)




if __name__ == '__main__':
    
    dataset = Im2LatexDataset()
    dataset.save("data_real2.pkl")


start make file dataset.pkl
make dataset


  0%|          | 0/22806 [00:00<?, ?it/s]

success!


In [ ]:
equal_pred = []
path_list = []
data_path_real =[]
data_path_gen = []

def make_data(start, end):
  with tqdm.tqdm(total=end-start) as pbar:
    for k in range(end-start):
      for paths in glob.glob(f"/content/datareal/1110/*_{k}.*"):
          path = paths.split("/")[-1]
          img = cv2.imread(paths)
          img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
          math = model(img, False)
          image = latex_to_png(math,backend="matplotlib", wrap=True)
          try:
              ig = Image.open(io.BytesIO(image))
              ig_new = Image.new('RGB', ig.size, color = 'white')
              ig_new.paste(ig,(0,0),ig)
          except:
              ig_new = Image.new('RGB', (img.shape[1], img.shape[0]), color = 'white')
      
          cv2.imwrite("/content/data_debug2/data_read_" + path, img)
          ig_new.save("/content/data_debug2/data_gen_" + path)
          # cv2.imwrite("./data_debug2/data_gen_" + path, ig_new)
      
          equal_pred.append(math)
          path_list.append(path)
          data_path_real.append("C:\\Users\\Tran Ngoc Du\\Downloads\\content\\data_debug2\\data_read_" + path)
          data_path_gen.append("C:\\Users\\Tran Ngoc Du\\Downloads\\content\\data_debug2\\data_gen_" + path)
          pbar.update(1)


make_data(1, 100)
save_frame = pd.DataFrame({"image":path_list, "equal_pred":equal_pred, "data_path_real":data_path_real, "data_path_gen":data_path_gen})
save_frame.to_excel("./debug.xlsx", index=False)


197it [00:48,  4.07it/s]


In [ ]:
%ls /usr/local/lib/python3.7/dist-packages/pix2tex/

api/      gui.py       models/       resources/        utils/
cli.py    __init__.py  models.py     setup_desktop.py
dataset/  __main__.py  __pycache__/  train.py
eval.py   model/       real_make.py  train_resizer.py


In [ ]:
!echo "Hello World" > /usr/local/lib/python3.7/dist-packages/pix2tex/real_make.py

#make real_data_bug

In [ ]:
%%writefile /usr/local/lib/python3.7/dist-packages/pix2tex/real_make.py
from pix2tex.dataset.dataset import Im2LatexDataset
import argparse
import logging
import yaml

import numpy as np
import torch
from torchtext.data import metrics
from munch import Munch
from tqdm.auto import tqdm
import wandb
from Levenshtein import distance
import pandas as pd
from pix2tex.models import get_model, Model
from pix2tex.utils import *



def detokenize(tokens, tokenizer):
    toks = [tokenizer.convert_ids_to_tokens(tok) for tok in tokens]
    for b in range(len(toks)):
        for i in reversed(range(len(toks[b]))):
            if toks[b][i] is None:
                toks[b][i] = ''
            toks[b][i] = toks[b][i].replace('Ġ', ' ').strip()
            if toks[b][i] in (['[BOS]', '[EOS]', '[PAD]']):
                del toks[b][i]
    return toks


@torch.no_grad()
def evaluate(model: Model, dataset: Im2LatexDataset, args: Munch, num_batches: int = None, name: str = 'test'):
    """evaluates the model. Returns bleu score on the dataset

    Args:
        model (torch.nn.Module): the model
        dataset (Im2LatexDataset): test dataset
        args (Munch): arguments
        num_batches (int): How many batches to evaluate on. Defaults to None (all batches).
        name (str, optional): name of the test e.g. val or test for wandb. Defaults to 'test'.

    Returns:
        Tuple[float, float, float]: BLEU score of validation set, normed edit distance, token accuracy
    """
    assert len(dataset) > 0
    device = args.device
    log = {}
    equal_pred = []
    data_path_real =[]
  
    pbar = tqdm(enumerate(iter(dataset)), total=len(dataset))
    for i, (seq, im, image_path) in pbar:
        if seq is None or im is None:
            continue
        #loss = decoder(tgt_seq, mask=tgt_mask, context=encoded)
        dec = model.generate(im.to(device), temperature=args.get('temperature', .2))
        pred = detokenize(dec, dataset.tokenizer)
        pred = [post_process(item) for item in token2str(dec, dataset.tokenizer)]
        equal_pred+=pred
        data_path_real +=image_path

        save_frame = pd.DataFrame({"equal_pred":equal_pred, "data_path_real":data_path_real})
        save_frame.to_csv("./debug_tem.csv", index=False)
        
    


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Test model')
    parser.add_argument('--config', default=None, help='path to yaml config file', type=str)
    parser.add_argument('-c', '--checkpoint', default="/content/drive/MyDrive/mixed_e08_step5587.pth", type=str, help='path to model checkpoint')
    parser.add_argument('-d', '--data', default='dataset/data/val.pkl', type=str, help='Path to Dataset pkl file')
    parser.add_argument('--no-cuda', action='store_true', help='Use CPU')
    parser.add_argument('-b', '--batchsize', type=int, default=10, help='Batch size')
    parser.add_argument('--debug', action='store_true', help='DEBUG')
    parser.add_argument('-t', '--temperature', type=float, default=.333, help='sampling emperature')
    parser.add_argument('-n', '--num-batches', type=int, default=None, help='how many batches to evaluate on. Defaults to None (all)')

    parsed_args = parser.parse_args()
    if parsed_args.config is None:
        with in_model_path():
            parsed_args.config = os.path.realpath('settings/config.yaml')
    with open(parsed_args.config, 'r') as f:
        params = yaml.load(f, Loader=yaml.FullLoader)
    args = parse_args(Munch(params))
    args.testbatchsize = parsed_args.batchsize
    args.wandb = False
    args.temperature = parsed_args.temperature
    logging.getLogger().setLevel(logging.DEBUG if parsed_args.debug else logging.WARNING)
    seed_everything(args.seed if 'seed' in args else 42)
    model = get_model(args)
    if parsed_args.checkpoint is None:
        with in_model_path():
            parsed_args.checkpoint = os.path.realpath('checkpoints/weights.pth')
    model.load_state_dict(torch.load(parsed_args.checkpoint, args.device))
    dataset = Im2LatexDataset().load(parsed_args.data)
    valargs = args.copy()
    valargs.update(batchsize=args.testbatchsize, keep_smaller_batches=True, test=True)
    dataset.update(**valargs)
    evaluate(model, dataset, args, num_batches=parsed_args.num_batches)


Overwriting /usr/local/lib/python3.7/dist-packages/pix2tex/real_make.py


In [ ]:
!python -m pix2tex.real_make -c '/content/drive/MyDrive/mixed_e08_step5587.pth' -d /content/data_real2.pkl

start make file dataset.pkl
100% 2333/2333 [13:17<00:00,  2.93it/s]


In [ ]:
%mkdir data_real_gen4

In [ ]:
from torchmetrics.functional.text.squad import PREDS_TYPE
import tqdm
equal_pred = []
path_list = []
data_path_real =[]
data_path_gen = []

def make_data(start, end):
  debug_frame = pd.read_csv("/content/debug_tem.csv")
  real_path = debug_frame["data_path_real"]
  pred = debug_frame["equal_pred"]
  with tqdm.tqdm(total=end-start) as pbar:
    for k in range(end-start):
          paths = real_path.iloc[k]
          path = paths.split("/")[-1]
          math = pred.iloc[k]
          image = latex_to_png(math,backend="matplotlib", wrap=True)
          try:
              ig = Image.open(io.BytesIO(image))
              ig_new = Image.new('RGB', ig.size, color = 'white')
              ig_new.paste(ig,(0,0),ig)
          except:
              ig_new = Image.new('RGB', (50, 50), color = 'white')
      
          # cv2.imwrite("/content/data_debug2/data_real_gen" + path, img)
          ig_new.save("/content/data_real_gen4/" + path)
      
          equal_pred.append(math)
          path_list.append(path)
          data_path_real.append("D:\\1110\\" + path)
          data_path_gen.append("D:\\data_real_gen4\\" + path)
          pbar.update(1)


make_data(15000, 22000)
save_frame = pd.DataFrame({"image":path_list, "equal_pred":equal_pred, "data_path_real":data_path_real, "data_path_gen":data_path_gen})
save_frame.to_excel("./debug4.xlsx", index=False)


100%|██████████| 7000/7000 [01:18<00:00, 89.03it/s]


In [ ]:
!zip -r /content/data_real_gen4.zip /content/data_real_gen4